#### Example with image
Swap test1.jpg to eg test4.jpg to see it detect multiple faces


Note:
Haar is better at detecting faces but almost 10x slower.
haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

In [12]:
#import required libraries 
import cv2 #import OpenCV library
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')

#load test image
testImg = cv2.imread('data/test4.jpg')
# convert the test image to gray image as opencv face detector expects gray images
gray_img = cv2.cvtColor(testImg, cv2.COLOR_BGR2GRAY)
#detect multiscale faces (some images may be closer to camera than others) images
faces = lbp_face_cascade.detectMultiScale(gray_img, scaleFactor=1.2, minNeighbors=5); 
# print the number of faces found
print('Faces found: ', len(faces))

# go over list of faces and draw them as rectangles on img
for (x, y, w, h) in faces:
    cv2.rectangle(testImg, (x, y), (x+w, y+h), (0, 255, 0), 2)

# convert gray image to RGB and show image
cv2.imshow('test_image', testImg) # show result img

# PRESS ESC TO EXIT
cv2.waitKey(0) == 27
cv2.destroyAllWindows()

Faces found:  7


#### Example with video
Careful with frame size!

In [15]:
#import required libraries 
import cv2 #import OpenCV library
#importing time library
import time 
# matplotlib
import matplotlib.pyplot as plt
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')
# NOW USING HAAR BCAUSE OF ACCURACY BUT TBP IS WAAY FASTER
haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')
# cam is the videofeed from video0
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 180)
# use uvcdynctrl -f to find out device frame formats
cam.set(cv2.CAP_PROP_FPS, 15)
    
while True: # loop bcause video
    (re, img) = cam.read() # Grab tuples from cam (True/False, Frame)
    # convert the test image to gray image as opencv face detector expects gray images
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detect faces
    faces = haar_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=4); 

    # go over list of faces and draw them as rectangles on img
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)

    # Show result
    cv2.imshow('test', img) # show result img
    time.sleep(0.2) #this works as a limiter

    
    # PRESS ESC TO EXIT
    if cv2.waitKey(1) == 27:
        break
cam.release()
cv2.destroyAllWindows()

#### Trial with crop
Attempting to follow the face in the frame, crashes when face nears edge bcause fram size < 0

In [ ]:
#import required libraries 
import cv2 #import OpenCV library
import time #importing time library
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')
# cam is the videofeed from video0
cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 180)
cam.set(cv2.CAP_PROP_FPS, 15) # use uvcdynctrl -f to find out device frame formats
    
while True: # loop bcause video
    (re, img) = cam.read() # Grab tuples from cam (True/False, Frame)
    # convert the test image to gray image as opencv face detector expects gray images
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detect faces
    faces = haar_face_cascade.detectMultiScale(gray_img, scaleFactor=1.1, minNeighbors=4); 

    # go over list of faces and draw them as rectangles on img
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        crop_img = img[y-45:y+45+h, x-80:x+80+w]
        cv2.imshow("cropped", crop_img)
    #cropping
    # Show result
    #cv2.imshow('test', img) # show result img
    #time.sleep(1) #this works as a limiter

    
    # PRESS ESC TO EXIT
    if cv2.waitKey(1) == 27:
        break
cam.release()
cv2.destroyAllWindows()

#### Fixed crash on nearing edge by hardcoding limits to y and x
Working script, lpb cascades are much faster than haar, but bad in this application (big FOV, small face)

In [6]:
#import required libraries 
import cv2 #import OpenCV library
import time #importing time library
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')

cam = cv2.VideoCapture(0) # cam is the videofeed from video0
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 180)
cam.set(cv2.CAP_PROP_FPS, 15) # use uvcdynctrl -f to find out device frame formats
t1 = time.time() # DEBUG: start timer for monitoring speed

# Function to crop video
def cropvideo(frame,faces):
    for (x,y,w,h) in faces:
        if y <=45: # dont crop when nearing top of frame
            y = 45
        if x<=80: # dont crop when nearing left of frame
            x= 80
        global crop_img
        crop_img = frame[y-45:y+45+h, x-80:x+80+w] # crop video to 1/4 size
        # TODO: Check multiple face detection
    return crop_img

# Function to Track Face
def facetrack(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = lbp_face_cascade.detectMultiScale(gray, 1.2, 5) # img,scaleFactor,minNeighbors
    for (x,y,w,h) in faces: # iterate through all found faces
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2) # draw a rectangle
    return faces

def main():
    # Program function
    while True:
        (ret, frame) = cam.read() # Grab tuples from cam (True/False, Frame)
        faces = facetrack(frame)
        crop_img = cropvideo(frame,faces)
        cv2.imshow("cropped", crop_img)
        t5 = time.time() # DEBUG: check how long processing takes
        #print("Frame displayed at " + str(round(t5-t1, 3)) + "s")

        # PRESS ESC TO EXIT
        if cv2.waitKey(50) == 27:
            break
    cam.release()
    cv2.destroyAllWindows()

#For starting the .py script
if __name__ == '__main__':
    main()

In [6]:
#import required libraries 
import cv2 #import OpenCV library
import time #importing time library
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')
# NOW USING HAAR BCAUSE OF ACCURACY BUT TBP IS WAAY FASTER
haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

cam = cv2.VideoCapture(0) # cam is the videofeed from video0
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 320)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 180)
cam.set(cv2.CAP_PROP_FPS, 15) # use uvcdynctrl -f to find out device frame formats
t1 = time.time() # DEBUG: start timer for monitoring speed


def main():
    w = cv2.CAP_PROP_FRAME_WIDTH
    h = cv2.CAP_PROP_FRAME_HEIGHT
    y = 45
    x= 80
    # Program function
    while True:
        (ret, frame) = cam.read() # Grab tuples from cam (True/False, Frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar_face_cascade.detectMultiScale(gray, 1.2, 5, 0, (0, 0)) # img,scaleFactor,minNeighbors, flags, min_size

        if (len(faces)):
            for (x,y,w,h) in faces: # iterate through all found faces
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2) # draw a rectangle
                if y <=45: # dont crop when nearing top of frame
                    y = 45
                if x<=80: # dont crop when nearing left of frame
                    x= 80
                crop_img = frame[y-45:y+45+h, x-80:x+80+w] # crop video to 1/4 size
                cv2.imshow("cropped", crop_img)
        else:
            crop_img = frame[y-45:y+45+h, x-80:x+80+w]
            cv2.imshow("cropped", crop_img)
        t5 = time.time() # DEBUG: check how long processing takes
        #print("Frame displayed at " + str(round(t5-t1, 3)) + "s")

        # PRESS ESC TO EXIT
        if cv2.waitKey(50) == 27:
            break
    cam.release()
    cv2.destroyAllWindows()

#For starting the .py script
if __name__ == '__main__':
    main()

#### Bigger input frame, better detection of small faces
Tried increasing frame size for more reliable detection (also bigger edges around frame)

In [10]:
#import required libraries 
import cv2 #import OpenCV library
import time #importing time library
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')
# NOW USING HAAR BCAUSE OF ACCURACY BUT LBP IS WAAY FASTER
haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

cam = cv2.VideoCapture(0) # cam is the videofeed from video0
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cam.set(cv2.CAP_PROP_FPS, 15) # use uvcdynctrl -f to find out device frame formats
t1 = time.time() # DEBUG: start timer for monitoring speed


def main():
    w = cv2.CAP_PROP_FRAME_WIDTH
    h = cv2.CAP_PROP_FRAME_HEIGHT
    y = 135
    x= 200
    # Program function
    while True:
        (ret, frame) = cam.read() # Grab tuples from cam (True/False, Frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar_face_cascade.detectMultiScale(gray, 1.2, 5, 0, (0, 0)) # img,scaleFactor,minNeighbors, flags, min_size

        if (len(faces)):
            for (x,y,w,h) in faces: # iterate through all found faces
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2) # draw a rectangle
                if y <=135: # dont crop when nearing top of frame
                    y = 135
                if x<=200: # dont crop when nearing left of frame
                    x= 200
                crop_img = frame[y-135:y+135+h, x-200:x+200+w] # crop video to 1/4 size
                cv2.imshow("cropped", crop_img)
        else:
            crop_img = frame[y-135:y+135+h, x-200:x+200+w]
            cv2.imshow("cropped", crop_img)
        t5 = time.time() # DEBUG: check how long processing takes
        #print("Frame displayed at " + str(round(t5-t1, 3)) + "s")

        # PRESS ESC TO EXIT
        if cv2.waitKey(50) == 27:
            break
    cam.release()
    cv2.destroyAllWindows()

#For starting the .py script
if __name__ == '__main__':
    main()

#### How to output video? Testing cv2.videoWriter
Testing cell for streaming to .avi file

In [9]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)

ret, frame = cap.read()
h, w, c = frame.shape
print(h, w, c)

fourcc = cv2.VideoWriter_fourcc('L','M','P','2')
out = cv2.VideoWriter('output.avi', fourcc, 30.0, (w, h))

while(True):
    ret, frame = cap.read()

    out.write(frame)

    frame = cv2.resize(frame, (0,0), fx=1, fy=1)
    cv2.imshow("Frame", frame)

    ch = cv2.waitKey(1)
    if ch & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

480 640 3


#### Included output to .avi (this avi can be used in OBS as media input!)
Latest script, includes outputting to .avi file in MPEG2 for reading to OBS
NOT WORKING - crop_img not the same size as out.

In [1]:
#import required libraries 
import cv2 #import OpenCV library
import time #importing time library
#load cascade classifier training file for lbpcascade
lbp_face_cascade = cv2.CascadeClassifier('data/lbpcascade_frontalface.xml')
# NOW USING HAAR BCAUSE OF ACCURACY BUT LBP IS WAAY FASTER
haar_face_cascade = cv2.CascadeClassifier('data/haarcascade_frontalface_alt.xml')

cam = cv2.VideoCapture(0) # cam is the videofeed from video0
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 960)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
cam.set(cv2.CAP_PROP_FPS, 15) # use uvcdynctrl -f to find out device frame formats
t1 = time.time() # DEBUG: start timer for monitoring speed

def main():
    w = cv2.CAP_PROP_FRAME_WIDTH
    h = cv2.CAP_PROP_FRAME_HEIGHT
    y = 135
    x= 200
    fourcc = cv2.VideoWriter_fourcc('L','M','P','2')
    out = cv2.VideoWriter('output.avi', fourcc, 30.0, (w, h))
    
    # Program function
    while True:
        (ret, frame) = cam.read() # Grab tuples from cam (True/False, Frame)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar_face_cascade.detectMultiScale(gray, 1.2, 5, 0, (0, 0)) # img,scaleFactor,minNeighbors, flags, min_size

        if (len(faces)):
            for (x,y,w,h) in faces: # iterate through all found faces
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2) # draw a rectangle
                if y <=135: # dont crop when nearing top of frame
                    y = 135
                if x<=200: # dont crop when nearing left of frame
                    x= 200
                crop_img = frame[y-135:y+135+h, x-200:x+200+w] # crop video to 1/4 size
                cv2.imshow("cropped", crop_img) # DEBUG: show cropped frame
                out.write(crop_img) # push frame to avi
        else:
            crop_img = frame[y-135:y+135+h, x-200:x+200+w]
            cv2.imshow("cropped", crop_img) # DEBUG: show cropped frame
            out.write(crop_img) # push frame to avi
        t5 = time.time() # DEBUG: check how long processing takes
        #print("Frame displayed at " + str(round(t5-t1, 3)) + "s")

        # PRESS ESC TO EXIT
        if cv2.waitKey(50) == 27:
            break
    cam.release()
    out.release()
    cv2.destroyAllWindows()

#For starting the .py script
if __name__ == '__main__':
    main()


#### TODO:
- Fix out frame size
- Fix hard-coded x,y,w,h to adapt to input video.
- Maybe some flow control, while True is pretty ugly
- Maybe separate functionality into functions (input_frame, detect_face, crop_img, stream_out)?